In [4]:
# import keras.callbacks
from music21 import *

import re
import xml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dense, Dropout
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


In [5]:
with open('environment.txt') as f:
    env = f.readlines()
# you may also want to remove whitespace characters like \n at the end of each line
env = [x.strip() for x in env]

us = environment.UserSettings()
us['musicxmlPath'] = env[0]
us['musescoreDirectPNGPath'] = env[0]



In [6]:
# fix random seed for reproducibility
np.random.seed(7)

In [7]:
bachRegex = re.compile(r'.*bwv.*', re.IGNORECASE)
bd = corpus.search(bachRegex)

In [8]:
def convertSongToDataFrame(song):
    df = pd.DataFrame()
    
    currentKeySignature = None
    currentTimeSignature = None
    for measure in song.measures(0, None, collect=('TimeSignature','KeySignature')):
        if measure.keySignature:
            currentKeySignature = measure.keySignature
        if measure.timeSignature:
            currentTimeSignature = measure.timeSignature.ratioString
        for i in measure.iter:
            if type(i) is chord.Chord:
                df2 = pd.DataFrame([currentKeySignature, currentTimeSignature, roman.romanNumeralFromChord(i, currentKeySignature).figure, i.duration.quarterLength, i.offset + measure.offset])
                df = df.append(df2[0], ignore_index = True)
    return df


In [9]:
# only run if you want to create csv files
import os.path
import os
if not os.path.exists('./data'):
    os.makedirs("./data")
for s in bd:
    fname = "./data/" + s.sourcePath.split("/")[1]+'.csv'
    if os.path.isfile(fname):
        continue
    df = convertSongToDataFrame(s.parse().chordify())
    df.to_csv(fname, sep=',')
    
#failed at bwv276.mxl

AttributeError: 'KeySignature' object has no attribute 'tonic'

In [113]:
from functools import reduce
scores = []
import glob, os
os.chdir("C:/Users/errpo/Desktop/projects/project folder/music_prediction/data")
for file in glob.glob("*.csv"):
    scores.append(file)
scoreDf = []

for i in scores:
    score = pd.read_csv(i,usecols=[1,2,3,4,5])
    keySignatures = pd.DataFrame([score['0'].astype('category').cat.codes.unique()], columns=score['0'].unique())
    timeSignatures = pd.DataFrame([score['1'].astype('category').cat.codes.unique()], columns=score['1'].unique())
    chords = pd.DataFrame([score['2'].astype('category').cat.codes.unique()], columns=score['2'].unique())
    score['0'] = score['0'].astype('category').cat.codes
    score['1'] = score['1'].astype('category').cat.codes
    score['2'] = score['2'].astype('category').cat.codes
    score.columns = ['keySignature', 'timeSignature', 'chord', 'duration', 'timestamp']
    scoreDf.append(score)
    
# print(scoreDf[1])
scores = pd.concat(scoreDf[0:10])
# scores = scoreDf[0:10]
testScores = pd.concat(scoreDf[11:12])

avgScoreLength = math.floor(reduce((lambda x,y: x+ len(y)), scoreDf, 0)/len(scoreDf))


In [120]:
values = scores.values
trainX, trainY = values[:-1, :], values[-1, :]


array([  0.,   0.,  20.,   3.,  77.])

In [69]:
#LSTM
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

In [101]:
examples = 20
y_examples = 5
nb_samples = len(data) - examples - y_examples

# np.atleast_2d(data[0:100,:])

In [102]:
input_list = [np.expand_dims(np.atleast_2d(data[i:examples+i,:]), axis=0) for i in range(nb_samples)]
#input_list = [np.atleast_2d(data[i:examples+i,:]) for i in range(nb_samples)]
input_mat = np.concatenate(input_list, axis=0)

input_mat.shape

(136, 20, 5)

In [103]:
target_list = [np.atleast_2d(data[i+examples:examples+i+y_examples,0]) for i in range(nb_samples)]
target_mat = np.concatenate(target_list, axis=0)


In [104]:
trials = input_mat.shape[0]
features = input_mat.shape[2]
hidden = 64
model = Sequential()
model.add(LSTM(hidden, input_shape=(examples, features)))
model.add(Dropout(.2))
# model.add(Flatten())
model.add(Dense(y_examples))
model.add(Activation('linear'))
model.compile(loss='mse', optimizer='rmsprop')

In [105]:
history = LossHistory()
fittedModel = model.fit(input_mat, target_mat, nb_epoch=100, batch_size=400, callbacks=[history])

c:\users\errpo\anaconda3\envs\music\lib\site-packages\keras\models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/100
136/136 [==============================] - 1s 9ms/step - loss: 0.6919
Epoch 2/100
136/136 [==============================] - 0s 221us/step - loss: 0.4556
Epoch 3/100
136/136 [==============================] - 0s 221us/step - loss: 0.3445
Epoch 4/100
136/136 [==============================] - 0s 251us/step - loss: 0.2754
Epoch 5/100
136/136 [==============================] - 0s 170us/step - loss: 0.2342
Epoch 6/100
136/136 [==============================] - 0s 236us/step - loss: 0.1932
Epoch 7/100
136/136 [==============================] - 0s 195us/step - loss: 0.1662
Epoch 8/100
136/136 [==============================] - 0s 199us/step - loss: 0.1504
Epoch 9/100
136/136 [==============================] - 0s 214us/step - loss: 0.1298
Epoch 10/100
136/136 [==============================] - 0s 273us/step - loss: 0.1211
Epoch 11/100
136/136 [==============================] - 0s 317us/step - loss: 0.1002
Epoch 12/100
136/136 [==============================] - 0s 195us/step - loss

Epoch 98/100
136/136 [==============================] - 0s 214us/step - loss: 0.0052
Epoch 99/100
136/136 [==============================] - 0s 188us/step - loss: 0.0052
Epoch 100/100
136/136 [==============================] - 0s 184us/step - loss: 0.0056


In [108]:
model.predict(input_mat)

array([[ -1.47216804e-02,  -2.82412414e-02,  -7.93770514e-03,
          1.78092476e-02,   4.48392443e-02],
       [  6.56460598e-02,  -3.88717279e-02,   2.37878095e-02,
          6.58905692e-03,   1.39569221e-02],
       [  4.49624360e-02,  -3.94404903e-02,  -3.23126838e-03,
          1.92739312e-02,   4.81510721e-02],
       [  6.99586868e-02,  -5.09761460e-03,   2.81620491e-02,
          2.33775172e-02,   3.20235174e-03],
       [ -9.70961340e-03,  -1.75395813e-02,  -1.32494792e-03,
          7.82436877e-03,   1.28160743e-02],
       [ -1.42607279e-03,  -1.93047337e-03,   5.11769019e-03,
          1.28774997e-02,   2.22667642e-02],
       [ -2.05284543e-03,  -3.79026830e-02,  -1.32412817e-02,
          1.70727465e-02,   9.08499863e-03],
       [ -1.59726385e-02,  -2.99021360e-02,  -9.04152170e-03,
          7.10047595e-03,   9.30537190e-03],
       [  4.13192511e-02,  -6.12267852e-03,  -1.84325129e-03,
         -5.41319139e-03,  -1.20686637e-02],
       [ -4.86851297e-03,  -2.7840917